<h1>SQL Injection</h1><p><img src="images/1line.png" width="100%/"  /></p>

<ul>
<li>Specifically in this lecture shows you how SQL Injection works. <em>Don&rsquo;t do this on any site that you don&rsquo;t own!</em></li>
</ul>
<h3>SQL Injection</h3>
<ul>
<li>SQL injection is a code injection technique, used to attack data-driven applications, in which malicious SQL statements are inserted into an entry field for execution (e.g. to dump the database contents to the attacker).[1]</li>
<li>SQL injection must exploit a security vulnerability in an application's software,
<ul>
<li>for example, when user input is either incorrectly filtered for string literal escape characters embedded in SQL statements:
<ul>
<li>Value delimiter:&nbsp; <span style="color: #0000ff;"><strong>'</strong></span>&nbsp; or<span style="color: #0000ff;"> <strong>"</strong></span></li>
<li>End of Statement:&nbsp; <span style="color: #0000ff;"><strong>;</strong></span></li>
<li>Inline Comment:&nbsp; <span style="color: #0000ff;"><strong>/* comment */</strong></span></li>
<li>Remainder of line Comment:&nbsp; <span style="color: #0000ff;"><strong>-- comment</strong></span></li>
</ul>
</li>
</ul>
</li>
<li>SQL injection is mostly known as an attack vector for websites but can be used to attack any type of SQL database.</li>
</ul>
<h4>Forms of SQL Injection include:</h4>
<ol>
<li>Incorrectly filtered escape characters:
<ul>
<li>The following line of code illustrates this vulnerability:<br />
<pre>qry = "SELECT * FROM users WHERE name ='" + userName + "';"
If the Hacker uses this as the username:
' or '1'='1
This is the query that will be run
SELECT * FROM users WHERE name = '' OR '1'='1';</pre>
</li>
</ul>
</li>
<li>Incorrect type handling</li>
</ol>
<h4>Defense against SQL Injection</h4>
<ol>
<ol>
<li>The proper defense against SQL injection, which should always be done, is either prepared statements or parameterized queries. However, let&rsquo;s consider what would happen if the developer had validated the input but still formed the query with string concatenation. In this case, the code might look like:</li>
</ol>
</ol>

In [2]:
#This code creates the users database.  
import sqlite3 

conn = sqlite3.connect('users.sqlite')
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS users(id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, account REAL)")

data = [('Jane',52642),
        ('Paul',57127),
        ('Steve', 9000),
        ('Vince', 29000),
        ('Ben', 350000),
        ('Ham', 41400),
        ('James', 21600)]

cur.executemany("INSERT INTO users (name, account) VALUES(?, ?)", data)

conn.commit()

In [3]:
# Example SQL Injection
userName = "' or '1'='1"
qry = "SELECT * FROM users WHERE name ='" + userName + "';"

for row in cur.execute(qry):
    print(row)

(1, 'Jane', 52642.0)
(2, 'Paul', 57127.0)
(3, 'Steve', 9000.0)
(4, 'Vince', 29000.0)
(5, 'Ben', 350000.0)
(6, 'Ham', 41400.0)
(7, 'James', 21600.0)


In [4]:
# Another SQL Injection
injection = "3 OR name is not NULL"

for row in cur.execute('SELECT * FROM users Where ID=' + injection):
    print(row)

(1, 'Jane', 52642.0)
(2, 'Paul', 57127.0)
(3, 'Steve', 9000.0)
(4, 'Vince', 29000.0)
(5, 'Ben', 350000.0)
(6, 'Ham', 41400.0)
(7, 'James', 21600.0)


In [5]:
# Using parameterized query to cause code to crash instead of allowing injection
injection = "3 OR name is not NULL"

for row in cur.execute('SELECT * FROM users Where ID=?', (injection)):
    print(row)


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 21 supplied.

In [6]:
# Using input validation to avoid program crash due to injection
def validate_user_input(text):
    illegal = [";", "\'", "\"", "\\", "*"]
    for char in illegal :
        if(text.find(char) >= 0): return False
    return True


injection = "Jane'; DROP TABLE IF EXISTS users; Select * from '"

if(validate_user_input(injection)):
    for row in cur.execute("SELECT * FROM users Where name='" + injection + "'"):
        print(row)
else:
    print("Invalid input")

Invalid input


<ol start="2"><li>Sanitize Input: For data types like user ids, phone numbers, quantities, email addresses, and many others, input validation would not have allowed the single quote, which has already stopped the attack. However, for a field like a last name, a single quote must be allowed or else O’Malley will not work.</li></ul>

In [7]:
# Using input validation AND parameterized quer to avoid program crash due to injection (BEST)
def validate_user_input(text):
    illegal = [";", "\'", "\"", "\\", "*"]
    for char in illegal :
        if(text.find(char) >= 0): return False
    return True


injection = "Jane'; DROP TABLE IF EXISTS users; Select * from '"

if(validate_user_input(injection)):
    for row in cur.execute("SELECT * FROM users Where name=?", (injection)):
        print(row)
else:
    print("Invalid input")

Invalid input


<ol start="3">
<li>Least Possible Privileges</li>
</ol>
<ul>
<li>&nbsp;&ldquo;A user (or process) should have the lowest level of privilege required to perform his assigned task&rdquo;</li>
<li>Use GRANT &ndash; create users / grant them privileges</li>
<li>Use REVOKE &ndash; remove privileges</li>
<li>Available Privileges:</li>
<ul>
<li>ALL</li>
<li>SELECT</li>
<li>INSERT, DELETE, UPDATE</li>
<li>CREATE, ALTER, DROP</li>
<li>USAGE //no privileges</li>
</ul>
</ul>
<ul>
<li>The DBUSER account used by your Python pages should probably have SELECT, INSERT and UPDATE Privileges only.</li>
<li>To reset privileges in a MySQL database:<br />
<pre>REVOKE ALL ON dbtest.* FROM dbuser
GRANT SELECT, INSERT, UPDATE ON dbtest.* TO dbuser IDENTIFIED BY 'xxx'</pre>
</li>
</ul>
<ol start="4">
<li>Make final field in your Query as short as possible to reduce the chances SQL injected code will be possible.</li>
</ol>
<ul>
<li>For example: If your last field in your query is the State and it is limited to 2 characters&nbsp; then the likelihood of successful SQL Injection&nbsp; goes down.</li>
<li>The lesson here is: Just because you validated the data does not mean that you can be sloppy elsewhere.</li>
</ul>
<p>&nbsp;</p><hr>
<h3>References:</h3>
<ul>
<li><a>http://www.newthinktank.com/2011/01/php-security-pt-2/</a></li>
</ul>